In [2]:
# dbutils.library.restartPython()

# Import de Funções e Libs

As bibliotecas e funções utilizadas ao longo do código devem ser adicionadas aqui

In [3]:
import numpy as np
import pandas as pd
import requests
import json
import sherlock
import regras_politica as rp

### Testes Modulares - Referente ao tópico 4 - Tabelas tópico 4.3

### Geração de Massa

In [4]:
#Gerando a massa modelo passando o caminho do JSON como nome da variável
base_entrada = pd.DataFrame({'payloadHomol.intermediarias.faixaScoreExterno':["R10"],
                             'payloadHomol.intermediarias.segmentacaoPolitica':["P1 - Funcionarios"],
                             'payloadHomol.intermediarias.faixaRendaLiquida':["d. >= 10k"],
                             'payloadHomol.intermediarias.admissaoMeses':[0],
                             'payloadHomol.intermediarias.cpf5e6Digito':[99]
                            })

In [5]:
#Criação das massas com casos forçados
massa_TabelaMarAberto = sherlock.Gera_Massa(base_entrada)
massa_TabelaMarAberto.gera_cenario('payloadHomol.intermediarias.faixaScoreExterno',["R1","R2","R3",
                                                                                            "R4","R5","R6",
                                                                                            "R7","R8","R9",
                                                                                            "R10"])
massa_TabelaMarAberto.gera_cenario('payloadHomol.intermediarias.segmentacaoPolitica',["N1 - FOPAG",
                                                                                      "N2.1 - Varejo",
                                                                                      "N2.2 - Varejo+",
                                                                                      "N2.3 - Alta Renda",
                                                                                      "N2.1 - Varejo"])
massa_TabelaMarAberto.gera_cenario('payloadHomol.intermediarias.faixaRendaLiquida',['a. Sem renda',
                                                                                  'b. < 4k',
                                                                                  'c. 4k - 10k',
                                                                                  'd. >= 10k',
                                                                                  'z. Verificar'])
massa_TabelaMarAberto.gera_cenario('payloadHomol.intermediarias.admissaoMeses',[0,1,2,3,4,5])
massa_TabelaMarAberto.gera_cenario('payloadHomol.intermediarias.cpf5e6Digito',[0,43,44,45,46,88,89,90,91,99])


massa_TabelaMarAberto.get_massa_final()

massa_TabelaMarAberto.set_bom_arqv()
massa_TabelaMarAberto.converte_massa()

--------------------------------------
Payload setado com sucesso
--------------------------------------
Conversao concluida com sucesso


In [6]:
massa_TabelaMarAberto.get_massa_final()

,payloadHomol.intermediarias.faixaScoreExterno,payloadHomol.intermediarias.segmentacaoPolitica,payloadHomol.intermediarias.faixaRendaLiquida,payloadHomol.intermediarias.admissaoMeses,payloadHomol.intermediarias.cpf5e6Digito
0,R1,N1 - FOPAG,a. Sem renda,0,0
1,R1,N1 - FOPAG,a. Sem renda,0,43
2,R1,N1 - FOPAG,a. Sem renda,0,44
3,R1,N1 - FOPAG,a. Sem renda,0,45
4,R1,N1 - FOPAG,a. Sem renda,0,46
...,...,...,...,...,...
14995,R10,N2.1 - Varejo,z. Verificar,5,88
14996,R10,N2.1 - Varejo,z. Verificar,5,89
14997,R10,N2.1 - Varejo,z. Verificar,5,90
14998,R10,N2.1 - Varejo,z. Verificar,5,91


### Execução Modular e Geração de CSV

In [7]:
#EXEC MULTI NUCLEOS
#listaResultado = rp.execucaoModularConcurrent(massa_TabelaMarAberto.get_massa_json(),rp.SubGruposMarAberto)

In [8]:
#EXEC NORMAL
listaResultado = rp.execucaoModular(massa_TabelaMarAberto.get_massa_json(),rp.SubGruposMarAberto)

KeyError: 'limiteTeto'

In [ ]:
print(json.dumps(listaResultado[0],indent = 4))

##### Valida se todas condições estão sendo abordadas

In [ ]:
# buscando um perfil de cliente para análise
for payload in listaResultado:
    if (payload['payloadHomol']['intermediarias']['faixaScoreExterno']  == "R1"
        and
        payload['payloadHomol']['intermediarias']['segmentacaoPolitica'] == "N1 - FOPAG"
        and 
        payload['payloadHomol']['intermediarias']['faixaRendaLiquida'] == "b. < 4k"
        and
        payload['payloadHomol']['intermediarias']['admissaoMeses'] == 1
        ):
        print(json.dumps(payload,indent = 4))
        break

In [ ]:
# mudando o max de display de linhas dos dataframes desse notebook
pd.set_option('display.max_rows', 300) 
import pandas as pd

# fazendo freq para analisar se todos os cenários da tabela são abordados na massa
results = []
for caso in listaResultado:
    results.append(caso['payloadHomol']['intermediarias']['cenario'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha.sort_values(by='valor', ascending=True)

In [ ]:
# Renomeação das colunas para o cabeçalho do RMA
massa_TabelaMarAberto.rename_cabecalho()

In [ ]:
massa_TabelaMarAberto.get_massa_dm()

In [ ]:
massa_TabelaMarAberto.atribui_expected(listaResultado,"payloadHomol.intermediarias.segmentacao","segmentacaoSubgrupo")
massa_TabelaMarAberto.atribui_expected(listaResultado,"payloadHomol.intermediarias.alavancagem","alavancagem")
massa_TabelaMarAberto.atribui_expected(listaResultado,"payloadHomol.intermediarias.limiteTeto","limiteTeto")
massa_TabelaMarAberto.atribui_expected(listaResultado,"payloadHomol.intermediarias.limitePiso","limitePiso")
massa_TabelaMarAberto.atribui_expected(listaResultado,"payloadHomol.intermediarias.limiteFixo","limiteFixo")
massa_TabelaMarAberto.atribui_expected(listaResultado,"payloadHomol.intermediarias.aprovadoBAUAux","aprovadoBAUAux")
massa_TabelaMarAberto.atribui_expected(listaResultado,"payloadHomol.intermediarias.aprovadoSLAux","aprovadoSmallLimitsAux")

massa_TabelaMarAberto.get_massa_dm()

### Conversão da massa para CSV

In [ ]:
massa_TabelaMarAberto.get_massa_dm().to_csv('C:/Users/T51657/Desktop/Homologação/Modelos Online/v2/massas/Modulares/massaDefinicaoTabelasSubGruposMarAberto.csv',index=False)